In [2]:
pip install groq psycopg2-binary python-dotenv


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
# API key 
from dotenv import load_dotenv
import os

load_dotenv("env.txt")

print("Loaded key:", os.getenv("GROQ_API_KEY"))


Loaded key: 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


In [13]:
# 

from dotenv import load_dotenv
import os
from groq import Groq

load_dotenv("env.txt")  # or ".env"

assert os.getenv("GROQ_API_KEY") is not None, "API key not loaded"

client = Groq(api_key=os.getenv("GROQ_API_KEY"))
print("Groq client ready")

Groq client ready


In [7]:
!pip install psycopg2-binary sqlalchemy pandas


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.1 MB ? eta -:--:--
   --------- ------------------------------ 0.5/2.1 MB 1.3 MB/s eta 0:00:02
   -------------- ------------------------- 0.8/2.1 MB 1.2 MB/s eta 0:00:02
   -------------- ------------------------- 0.8/2.1 MB 1.2 MB/s eta 0:00:02
   -------------- ------------------------- 0.8/2.1 MB 1.2 MB/s eta 0:00:02
   ------------------- -------------------- 1.0/2.1 MB 751.1 kB/s eta 0:00:02
   ------------------- -------------------- 1.0/2.1 MB 751.1 kB/s eta 0:00:02
   ------------------- -------------------- 1.0/

In [5]:
# Establishing connection between Jupyter notebbok and PostgreSQL

from sqlalchemy import create_engine
from sqlalchemy.engine import URL
import pandas as pd

url = URL.create(
    drivername="postgresql+psycopg2",
    username="postgres",
    password="Qwerty@123",
    host="localhost",
    port=5432,
    database="nl2sql_SuperStore"
)

engine = create_engine(url)


In [6]:
pd.read_sql("SELECT 1;", engine)

,?column?
0,1


In [7]:
# Verify tables
pd.read_sql("""   
SELECT table_name
FROM information_schema.tables
WHERE table_schema = 'public';
""", engine)


,table_name
0,superstore
1,customer
2,product
3,orders
4,superstore_analytics


In [8]:
pd.read_sql("""
SELECT category, SUM(sales) AS total_sales
FROM superstore_analytics
GROUP BY category;
""", engine)


,category,total_sales
0,Furniture,741999.98
1,Office Supplies,719046.99
2,Technology,836154.10


In [20]:
SCHEMA_PROMPT = """
You are an expert PostgreSQL data analyst.

Database schema:

customers(
    customer_id PRIMARY KEY,
    customer_name,
    segment,
    country,
    city,
    state,
    region
)

products(
    product_id PRIMARY KEY,
    category,
    sub_category,
    product_name
)

orders(
    order_id,
    order_date,
    ship_date,
    customer_id FOREIGN KEY REFERENCES customers(customer_id),
    product_id FOREIGN KEY REFERENCES products(product_id),
    sales,
    quantity,
    discount,
    profit
)

Rules:
- Use only these tables
- Use proper JOINs
- Do not explain SQL
- Return only SQL
"""


In [15]:
#  NL → SQL Function
def nl_to_sql(user_question):
    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        temperature=0,
        messages=[
            {"role": "system", "content": SCHEMA_PROMPT},
            {"role": "user", "content": user_question}
        ]
    )
    return response.choices[0].message.content.strip()


In [16]:
question = "Total sales by category"
sql_query = nl_to_sql(question)
print(sql_query)


SELECT category, SUM(sales) as total_sales FROM superstore GROUP BY category;


In [17]:
# 
import pandas as pd

df = pd.read_sql(sql_query, engine)
df

,category,total_sales
0,Furniture,741999.98
1,Office Supplies,719046.99
2,Technology,836154.10


In [18]:
# LLMs can fail sometimes. Add this protection now
def run_nl2sql(question):
    try:
        sql = nl_to_sql(question)
        print("Generated SQL:\n", sql)
        return pd.read_sql(sql, engine)
    except Exception as e:
        print("Error executing SQL:", e)


In [19]:
 # Test this
run_nl2sql("Average profit by region")


Generated SQL:
 SELECT region, AVG(profit) AS avg_profit FROM superstore GROUP BY region;


,region,avg_profit
0,South,28.857846
1,West,33.849138
2,East,32.135829
3,Central,17.092746


In [21]:
question = "Total sales by category"
sql_query = nl_to_sql(question)

print(sql_query)


```sql
SELECT 
    p.category, 
    SUM(o.sales) AS total_sales
FROM 
    orders o
JOIN 
    products p ON o.product_id = p.product_id
GROUP BY 
    p.category
ORDER BY 
    total_sales DESC;
```
